In [2]:
import asyncio
import aiofiles
import glob
import json
import csv
import os
import random
import pandas as pd

In [ ]:
semaphore = asyncio.Semaphore(50)
async def read_json_file(file):
    """Read JSON files and extract required fields."""
    async with semaphore:
        async with aiofiles.open(file, "r", encoding="utf-8") as f:
            try:
                data = json.loads(await f.read())
                try:
                    results=data.get("results", [])
                    # processed_data = []
                    # for res in results:
                    #     try:
                    #         station_id = res["id"]
                    #         name = res["name"]
                    #         latitude = res["latitude"]
                    #         longitude = res["longitude"]
                    #         geom = f"SRID=4326;POINT({longitude} {latitude})"
                    #         processed_data.append((station_id, name, geom, latitude, longitude))
                    #     except KeyError:
                    #         print(f"⚠️ Missing fields in {file}: {res}")
                
                    return results

                except KeyError:
                    print(f"⚠️ Missing 'results' field in {file}")
                    return None

            except json.JSONDecodeError:
                print(f"❌ Error parsing {file}")
                return None
async def write_json_file(file, data):
    """Write JSON files asynchronously."""
    async with semaphore:
        async with aiofiles.open(file, "w", encoding="utf-8") as f:
            await f.write(json.dumps(data, indent=4, ensure_ascii=False))
            print(f"✅ {file} written.")
            return True

async def process_all_files(files,output_file):
    """Process all JSON files asynchronously."""
    tasks = [read_json_file(file) for file in files]
    results = await asyncio.gather(*tasks)
    all_results = [item for sublist in results for item in sublist]
    await write_json_file(output_file, all_results)
    return all_results



async def main():
    all_json_files = glob.glob('./api_responses/*.json')
    print("{} JSON files found.".format(len(all_json_files)))
    results=await process_all_files("all_stations.json",all_json_files)
    with open('all_stations.json', 'r') as f:
        data=json.load(f)
        df=pd.DataFrame(data)
        print (df.head())
        print (df.shape())
    



await main()


1542 JSON files found.
✅ all_stations.json written.


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

